In [ ]:
import nonlinear_benchmarks
import jax
import jax.numpy as jnp
#import orbax
#import orbax.checkpoint
import torch
from pathlib import Path
from jaxid.models import StateUpdateAndOptput, MLP
import jaxid.metrics
import matplotlib.pyplot as plt
from argparse import Namespace

In [ ]:
ckpt = torch.load("ckpt.pt")
#orbax.checkpoint.PyTreeCheckpointer().restore(
#    Path(".").resolve() / "models" / "model1"
#)
cfg = ckpt["cfg"]
cfg = Namespace(**cfg)

In [ ]:
f_xu = MLP(cfg.hidden_f + [cfg.nx])
g_x = MLP(cfg.hidden_g + [cfg.ny])
fg = StateUpdateAndOptput(f_xu, g_x)

In [ ]:
# Load test data
train_val, test = nonlinear_benchmarks.WienerHammerBenchMark()
u_test, y_test = test
u_test = u_test.reshape(-1, 1)
y_test = y_test.reshape(-1, 1)

In [ ]:
# Load scalers
params = ckpt["params"]
scaler_u = ckpt["scaler_u"]
scaler_y = ckpt["scaler_y"]

In [ ]:
# Define loss
def simulate(params, x0, u_seq):
    fg_func = lambda x, u: fg.apply(params, x, u)
    return jax.lax.scan(fg_func, x0, u_seq)

In [ ]:
u_test = scaler_u.transform(u_test)
x0 = jnp.zeros(cfg.nx)
_, y_test_hat = simulate(params, x0, u_test)
y_test_hat = scaler_y.inverse_transform(y_test_hat)

In [ ]:
plt.plot(y_test, "k")
plt.plot(y_test_hat, "b")
plt.plot(y_test_hat - y_test, "r")
plt.show()

In [ ]:
plt.plot(ckpt["LOSS"])

In [ ]:
fit = jaxid.metrics.fit_index(y_test, y_test_hat)[0]
rmse = jaxid.metrics.rmse(y_test, y_test_hat)[0] * 1000
nrmse = jaxid.metrics.nrmse(y_test, y_test_hat)[0]

print(f"{fit=} \n{rmse=} mV \n{nrmse=}")